In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from src.utils import make_features
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib,os,requests

In [ ]:
tickers=['AAPL','MSFT','SPY']
df=[]
for t in tickers:
    d=yf.download(t,period='5y',interval='1d',auto_adjust=True,progress=False)
    d['Ticker']=t
    df.append(d)
df=pd.concat(df).reset_index().rename(columns={'Date':'Date'})
df=df[['Date','Ticker','Open','High','Low','Close','Volume']]
df.to_csv('data/prices.csv',index=False)
d=df[df['Ticker']=='SPY'].set_index('Date')

In [ ]:
X,y=make_features(d)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=False)
m=LogisticRegression(max_iter=1000)
m.fit(X_train,y_train)
import sklearn.metrics as M
pred=m.predict(X_test)
proba=m.predict_proba(X_test)[:,1]
print(M.classification_report(y_test,pred))
os.makedirs('model',exist_ok=True)
joblib.dump(m,'model/model.pkl')

In [ ]:
m=joblib.load('model/model.pkl')
print(m.predict_proba(X.iloc[[-1]])[0,1])

In [ ]:
url='http://127.0.0.1:5000/predict'
payload={'features':{'ret':0.01,'vol':0.02,'ma5':100,'ma10':101,'mom5':0.03,'v_ma5':1000000}}
try:
    r=requests.post(url,json=payload,timeout=2)
    print(r.status_code,r.text)
except Exception as e:
    print('API未启动')